In [13]:
#Lapsed donors on this dataset are from the promotion nº 2, 17NK (from Forums)
data['RFA_2R'].value_counts()

L    95412
Name: RFA_2R, dtype: int64

# Data Mining Project

## Import and assessing data

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from math import ceil
import os

In [3]:
#goes to the previous folder, accesses the data folder, and gets the path to the donors.csv file (this is done bc the is
#too large to upload on GitHub)
path = os.path.join('..', 'data', 'donors.csv')

data = pd.read_csv(path, sep = ',')

data.head()

,Unnamed: 0,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,NOEXCH,...,AVGGIFT,CONTROLN,HPHONE_D,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A,GEOCODE2
0,0,2009-01-01,GRI,0,IL,61081,,,1957-12-01,0,...,7.741935,95515,0,L,4,E,X,X,X,C
1,1,2014-01-01,BOA,1,CA,91326,,,1972-02-01,0,...,15.666667,148535,0,L,2,G,X,X,X,A
2,2,2010-01-01,AMH,1,NC,27017,,,NaN,0,...,7.481481,15078,1,L,4,E,X,X,X,C
3,3,2007-01-01,BRY,0,CA,95953,,,1948-01-01,0,...,6.812500,172556,1,L,4,E,X,X,X,C
4,4,2006-01-01,,0,FL,33176,,,1940-01-01,0,...,6.864865,7112,1,L,2,F,X,X,X,A


In [11]:
# Get a copy of the original dataset
data_original = data.copy()

In [4]:
# Optional -> to check all the rows and columns
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [6]:
# Dimension of the dataset
data.shape

(95412, 476)

In [7]:
# Data types of the variables and columns
data.dtypes

Unnamed: 0     int64
ODATEDW       object
OSOURCE       object
TCODE          int64
STATE         object
               ...  
RFA_2A        object
MDMAUD_R      object
MDMAUD_F      object
MDMAUD_A      object
GEOCODE2      object
Length: 476, dtype: object

In [5]:
# Summary Statistics of our variables
data.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Unnamed: 0,95412,NaN,NaN,NaN,47705.5,27543.2,0,23852.8,47705.5,71558.2,95411
ODATEDW,95412,54,2015-01-01,15358,NaN,NaN,NaN,NaN,NaN,NaN,NaN
OSOURCE,95412,896,MBC,4539,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCODE,95412,NaN,NaN,NaN,54.2231,953.844,0,0,1,2,72002
STATE,95412,57,CA,17343,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
RFA_2A,95412,4,F,46964,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MDMAUD_R,95412,5,X,95118,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MDMAUD_F,95412,4,X,95118,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MDMAUD_A,95412,5,X,95118,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Info on the donors dataset
data.info(max_cols=476)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95412 entries, 0 to 95411
Data columns (total 476 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  95412 non-null  int64  
 1   ODATEDW     95412 non-null  object 
 2   OSOURCE     95412 non-null  object 
 3   TCODE       95412 non-null  int64  
 4   STATE       95412 non-null  object 
 5   ZIP         95412 non-null  object 
 6   MAILCODE    95412 non-null  object 
 7   PVASTATE    95412 non-null  object 
 8   DOB         71529 non-null  object 
 9   NOEXCH      95412 non-null  object 
 10  RECINHSE    95412 non-null  object 
 11  RECP3       95412 non-null  object 
 12  RECPGVG     95412 non-null  object 
 13  RECSWEEP    95412 non-null  object 
 14  MDMAUD      95412 non-null  object 
 15  DOMAIN      95412 non-null  object 
 16  HOMEOWNR    95412 non-null  object 
 17  CHILD03     95412 non-null  object 
 18  CHILD07     95412 non-null  object 
 19  CHILD12     95412 non-nu

In [9]:
# Check duplicates
any(data.duplicated())

False

In [10]:
# Count of missing values
data.isna().sum()

Unnamed: 0      0
ODATEDW         0
OSOURCE         0
TCODE           0
STATE           0
             ... 
RFA_2A          0
MDMAUD_R        0
MDMAUD_F        0
MDMAUD_A        0
GEOCODE2      132
Length: 476, dtype: int64

## Data Treatment

In [13]:
# Ver no coherence:
# O LASTDATE é o resumo das campanhas, mas como a ultima data das campanhas está, várias vezes, incoerente com as RDATE, e
# e tendo em conta que estas têm muitos missing values (algumas mais de 70%), decidimos dropar as RDATE. Assumimos, tbm, 
# que há muitos valores que podem ter sido inseridos manualmente, o que leva às incoerências.

In [14]:
##########################################################################################################
#Em principio, dropar as RAMNT; ADATE; RDATE, pq está tudo incoerente com as summary variables (checkar o código que tá no data_treatment_v2)
##########################################################################################################

In [15]:
#CONTROLN: stays as index of the dataset
data.set_index('CONTROLN', inplace=True)

In [16]:
#Problema pa depois:

#put aside the variables that are already related to a PVA's clustering, and may be used in the end to compare it to our
#solution...:
donors_pva_cluster = data[['MDMAUD', 'DOMAIN', 'GEOCODE', 'PEPSTRFL']].copy()

#MDMAUD: RFA field, already a clustering from the organization

#put this aside because it is clustering (this one is a bit different from the RFA variables)
#donors_pva_cluster.replace({'DOMAIN' : {' ' : np.nan}}, inplace = True)

#GEOCODE is what? Put aside for now - clustering
#donors_pva_cluster.replace({'GEOCODE' : {' ' : np.nan}}, inplace = True)

#probably put aside! bc it's related to RFA
#donors_pva_cluster.replace({'PEPSTRFL' : {'X' : '1', ' ' : '0'}}, inplace = True)

rfa1 = data.loc[:, 'RFA_2':'RFA_24'].copy()
donors_pva_cluster = pd.merge(donors_pva_cluster, rfa1, left_index=True, right_index=True)
#from RFA_2 to RFA_24: these are variables with a clustering solution from PVA; don't use them for now (maybe use them
#in the end to compare with our own clustering solution)

rfa_and_mdmaud = data.loc[:, 'RFA_2R':'MDMAUD_A'].copy()
donors_pva_cluster = pd.merge(donors_pva_cluster, rfa_and_mdmaud, left_index=True, right_index=True)
#RFA_2R, RFA_2F, RFA_2A are related to RFA (put aside for now?)
#MDMAUD_R, MDMAUD_F, MDMAUD_A are related to RFA (put aside for now?)

donors_pva_cluster.head()

,MDMAUD,DOMAIN,GEOCODE,PEPSTRFL,RFA_2,RFA_3,RFA_4,RFA_5,RFA_6,RFA_7,...,RFA_21,RFA_22,RFA_23,RFA_24,RFA_2R,RFA_2F,RFA_2A,MDMAUD_R,MDMAUD_F,MDMAUD_A
CONTROLN,,,,,,,,,,,,,,,,,,,,,
95515,XXXX,T2,,X,L4E,S4E,S4E,S4E,S4E,S4E,...,S4E,S4E,S4E,S4E,L,4,E,X,X,X
148535,XXXX,S1,02,,L2G,A2G,A2G,A2G,A2G,A1E,...,N1E,N1E,,F1E,L,2,G,X,X,X
15078,XXXX,R2,,X,L4E,S4E,S4E,S4E,S4E,S4F,...,,S4D,S4D,S3D,L,4,E,X,X,X
172556,XXXX,R2,,X,L4E,S4E,S4E,S4E,S4E,S4E,...,A1D,A1D,,,L,4,E,X,X,X
7112,XXXX,S2,,,L2F,A2F,A2F,A2F,A1D,I2D,...,A3D,I4E,A3D,A3D,L,2,F,X,X,X


In [18]:
#drop these:
data.drop(columns = ['Unnamed: 0',
                     'NOEXCH',
                     'WEALTH1',
                     'DATASRCE',
                     'LIFESRC'], inplace = True)

#Unnamed: 0: it has indexes from 0 to 95412, it's redundant

#NOEXCH: this var. has inconsistencies with the metadata available, as there's no reference to '0', '1', 0 or 1 values
#BELO - COHERENCE
#drop this! (incoherent, bc the variable has many incoherences with the metadata -> explicar melhor no relatório)
#data['NOEXCH'].value_counts()

#WEALTH1 has similar values to WEALTH2 but the metadata does not specify what is the variable, so we decided to keep
#the WEALTH2, which is well described
#data = data.astype({"WEALTH1": object})
#data['WEALTH1'].unique()

#data['DATASRCE'].unique()
#data.replace({'DATASRCE' : {' ' : np.nan}}, inplace = True)
#says which was the source of the data in this section of variables; it has 21280 ' ' (blanks)
#DROP! bc the variable is not very useful for interpreting the clusters (we are not sure about what are the values 
#representing... companies? we could not find on the internet)

#LIFESRC: same as DATASRCE...
#data['LIFESRC'].value_counts()
#data.replace({'LIFESRC' : {' ' : np.nan}}, inplace = True)
#says which was the source of the data in this section of variables; it has 54032 ' ' (blanks)
#DROP! bc the variable is not very useful for interpreting the clusters (we are not sure about what are the values 
#representing... companies? we could not find on the internet)

#We decided to drop all ADATE_x; RDATE_x and RAMNT_x bc they are incoherent with the summary variables.
#The code showing why we think these are incoherent is on the Coherence Check phase.

# Drop all 'ADATE_x'
data = data.drop(columns = data.loc[:,'ADATE_2':'ADATE_24'].columns)

# Drop all 'RDATE_x' and all 'RAMNT_x'
data = data.drop(columns = data.loc[:,'RDATE_3':'RAMNT_24'].columns)

In [19]:
#drop variables on the extra dataframe that has the clustering from PVA (donors_pva_cluster)
data.drop(columns = donors_pva_cluster.columns, inplace = True)

In [21]:
#note -> 'NUMCHLD' : int, altough it cannot be converted due to NaNs (float)!
data = data.astype({'TCODE': object, 
                    'INCOME': object,
                    'WEALTH2': object,
                    'MSA': object,
                    'ADI': object,
                    'DMA': object,
                    'HPHONE_D': object})

#INCOME: from the Forums: 7 corresponds to the highest income group and 0 to the lowest one

#MSA has values 0, which is not a MSA code. After researching what does this code represent in the US census, we assume 
#those 0s are for the doners that do not live in a metropolitan statistical area (MSA), a region with at least one 
#urbanized area and a minimum population of 50 000 (Investopedia); so those people may live in towns or rural areas.

#ADI has values 0, which is not an ADI code. After researching what does this code represents, we assume 
#those 0s are for the doners that do not live in a radio or television market region in which that specific population 
#receives the same programming (https://www.insightsassociation.org/issues-policies/glossary/area-dominant-influence-adi);
#this is related to marketing, and those people may live in regions that are not assigned to a code (but this is not a 
#missing value!).

#DMA has values 0, which is not a DMA code. After researching what does this code represents, we assume 
#those 0s are for the doners that do not live in a geographic area in the United States in which local television viewing 
#is measured by Nielsen (https://www.nielsen.com/us/en/intl-campaigns/dma-maps/#:~:text=DMA%20(Designated%20Market%20Area)%20regions,geographic%20areas%20within%20their%20business.)
#again, this code is related to marketing, and those people may live in regions that are not assigned to a code (but this 
#is not a missing value!).

#HPHONE_D is binary (0 or 1)

In [22]:
#dates as string, now as datetimes
#note -> errors='coerce' as in the vast majority of records (if not always) the errors on the to_datetime function are
#rased because the string in the column is empty, thus there isn't a date there; this leads to an error, and with this
#argument definition, the invalid parsing will be set as NaT (Not a Time), which is what we intend (missing values stay
#missing)
data['ODATEDW'] = pd.to_datetime(data['ODATEDW'])
data['DOB'] = pd.to_datetime(data['DOB'], errors='coerce')
#data.loc[:, 'ADATE_2':'ADATE_24'] = data.loc[:, 'ADATE_2':'ADATE_24'].apply(pd.to_datetime, errors='coerce') #from ADATE_2 until ADATE_24
data['MAXADATE'] = pd.to_datetime(data['MAXADATE'])
#data.loc[:, 'RDATE_3':'RDATE_24'] = data.loc[:, 'RDATE_3':'RDATE_24'].apply(pd.to_datetime, errors='coerce') #from RDATE_3 until RDATE_24
data['MINRDATE'] = pd.to_datetime(data['MINRDATE'])
data['MAXRDATE'] = pd.to_datetime(data['MAXRDATE'])
data['LASTDATE'] = pd.to_datetime(data['LASTDATE'])
data['FISTDATE'] = pd.to_datetime(data['FISTDATE'], errors='coerce')
data['NEXTDATE'] = pd.to_datetime(data['NEXTDATE'], errors='coerce')

In [23]:
#to remove the hyphen from the end of the ZIP codes (US zip codes are not supposed to have hyphens, but some do!)
data['ZIP'] = data['ZIP'].map(lambda x: x.replace('-', ''))

#now, the replace: on some vars. swap a blank space for a NaN, on binaries swap the letters for '0' and '1', etc
data.replace({'OSOURCE' : {' ' : np.nan},
                'MAILCODE' : {' ' : '0', 'B' : '1'},
                'PVASTATE' : {' ' : 'N'}, #N = state is not served by any organization
                'RECINHSE' : {' ' : '0', 'X' : '1'},
                'RECP3' : {' ' : '0', 'X' : '1'},
                'RECPGVG': {' ' : '0', 'X' : '1'},
                'RECSWEEP': {' ' : '0', 'X' : '1'},
                'HOMEOWNR' : {' ' : '0', 'H' : '1', 'U' : '0'}, #we assume the blank space on this variable says that it is unknown wether the donor has or not a home (so, we joined it with the 0-Unkwown)
                'CHILD03' : {' ' : 'N'}, #the ' ' (blank) will be replaced by 'N' (N = None), as we assume the blank space in this variable means the donor has no 
                'CHILD07' : {' ' : 'N'}, #children in that specific range of ages
                'CHILD12': {' ' : 'N'},  #note: more than 90 000 records have None, on each of these 4 variables
                'CHILD18': {' ' : 'N'},
                'NUMCHLD': {np.nan : 0}, #assuming NaN is zero, bc there is no zero on this variable!?
                'GENDER' : {' ' : 'U', 'A' : 'U', 'C' : 'U'}, #Incoherence (Belo!) -> nem que se fale só no relatório!
                                                             #values 'A' (2 of them) & 'C' (2) are not described on the metadata, most probably those are errors
                                                             #we assume the blank spaces, As and Cs can be assumed to be Unkwown
                'SOLP3' : {' ' : 'Default'},
                'SOLIH' : {' ' : 'Default'}, #feature engineering: create a binary with -> 0 (can't be mailed), 1 (can be mailed)
                'MAJOR' : {' ' : '0', 'X' : '1'},
                'GEOCODE2' : {' ' : np.nan}}, inplace = True)

#assuming Y = Yes and N = No, on all these vars
data.loc[:, 'COLLECT1':'PLATES'] = data.loc[:, 'COLLECT1':'PLATES'].replace({'Y' : '1', ' ' : '0'}) #from COLLECT1 to PLATES

In [39]:
#after the previous cell, we can put NUMCHLD as int:
data = data.astype({'NUMCHLD': int})

data['NUMCHLD'].unique()

array([0, 1, 2, 3, 4, 5, 6, 7])

In [26]:
#PROBLEM on HHP1 and HHP2: "Median Person Per Household" and "Average Person Per Household" but the values are bery high
#if we interpret this as the median/average number of people living in a household of the neighborhood, then it should
#have values between 1, 2 or 3. Not 100-700. (Incoherence - Belo?)



#perguntar ao professor!!
#decisão final: (nota: ainda há alguns valores iguais a 0)
data['HHP1'] = data['HHP1'] / 100
data['HHP2'] = data['HHP2'] / 100

In [27]:
#RHP1 to RHP4 are kinda weird (Housing Unit??)



#perguntar ao professor: média/mediana das pessoas ser tão alto (50?); ou a descrição da variável não está correta?
#decisão final: we assume this values were multiplied by 10 to avoid storing floats (floats make up more space on data
#files), so we divided this columns by 10, for the values to make more "sense"
data['RHP1'] = data['RHP1'] / 10
data['RHP2'] = data['RHP2'] / 10
data['RHP3'] = data['RHP3'] / 10
data['RHP4'] = data['RHP4'] / 10

In [28]:
#EC1: "Median Years of School Completed by Adults 25+" has minimum 89 and maximum 170... are the values wrong?

#HC1: "Percent Median Length of Residence" is percent and median at the same time? And has values of 0 length! Is it the 
#median time someone has been living in their current home, in years? Very ambiguous:
#HC2: this one has a less ambiguous description but it still says "Percent Median" so is it a percentage or a median? 
#it is confusing


#perguntar ao professor: EC1 (mediana de 170 anos a estudar?):
#decisão final: divided by 10, because 8.9 (min) and 17.0 (max) are values that make more sense
#we assume this values were multiplied by 10 to avoid storing floats (floats make up more space on data
#files), so we divided this columns by 10, for the values to make more "sense"
data['EC1'] = data['EC1'] / 10
#HC1 e HC2 -> we assume this is a median, the "percent" is probably a typo on the metadata descriptions

After the Data Treatment:

In [29]:
data.head()

,ODATEDW,OSOURCE,TCODE,STATE,ZIP,MAILCODE,PVASTATE,DOB,RECINHSE,RECP3,...,MAXRAMNT,MAXRDATE,LASTGIFT,LASTDATE,FISTDATE,NEXTDATE,TIMELAG,AVGGIFT,HPHONE_D,GEOCODE2
CONTROLN,,,,,,,,,,,,,,,,,,,,,
95515,2009-01-01,GRI,0,IL,61081,0,N,1957-12-01,0,0,...,12.0,2014-02-01,10.0,2015-12-01,2009-11-01,2010-03-01,4.0,7.741935,0,C
148535,2014-01-01,BOA,1,CA,91326,0,N,1972-02-01,0,0,...,25.0,2015-12-01,25.0,2015-12-01,2013-10-01,2015-04-01,18.0,15.666667,0,A
15078,2010-01-01,AMH,1,NC,27017,0,N,NaT,0,0,...,16.0,2012-07-01,5.0,2015-12-01,2010-01-01,2011-01-01,12.0,7.481481,1,C
172556,2007-01-01,BRY,0,CA,95953,0,N,1948-01-01,0,0,...,11.0,2014-11-01,10.0,2015-12-01,2007-02-01,2007-11-01,9.0,6.812500,1,C
7112,2006-01-01,NaN,0,FL,33176,0,N,1940-01-01,1,1,...,15.0,2016-01-01,15.0,2016-01-01,1999-03-01,2000-05-01,14.0,6.864865,1,A


In [30]:
data.info(max_cols=370)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 95412 entries, 95515 to 185114
Data columns (total 370 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   ODATEDW   95412 non-null  datetime64[ns]
 1   OSOURCE   94484 non-null  object        
 2   TCODE     95412 non-null  object        
 3   STATE     95412 non-null  object        
 4   ZIP       95412 non-null  object        
 5   MAILCODE  95412 non-null  object        
 6   PVASTATE  95412 non-null  object        
 7   DOB       71529 non-null  datetime64[ns]
 8   RECINHSE  95412 non-null  object        
 9   RECP3     95412 non-null  object        
 10  RECPGVG   95412 non-null  object        
 11  RECSWEEP  95412 non-null  object        
 12  HOMEOWNR  95412 non-null  object        
 13  CHILD03   95412 non-null  object        
 14  CHILD07   95412 non-null  object        
 15  CHILD12   95412 non-null  object        
 16  CHILD18   95412 non-null  object        
 17  NUMCHL

In [31]:
#'HIT' is good to go

In [32]:
#from MBCRAFT to PUBOPP, it all looks good; but it has many missing values!

In [33]:
#from MALEMILI to FEDGOV there are no MVs and everything is fine (check for incoherences: percentages below 0 or 
#above 100?)

In [34]:
#from POP901 until the end of this set of variables, we assume 0s in the first three (Number of Persons, Families and 
#Households) represent people that live in rural areas and small towns, with no one living near them (in the neighborhood)

#gets resolved when we do data = data[data['AGE902'] != 0] (Coherence checks!)

In [35]:
#from POP901 until ETH16, everything looks fine

In [26]:
#PROBLEM on HV1 & HV2: "Median Home Value in hundreds" has values of 0, and then, the second lowest, 75 (which means 7500) 
#the values of 0 are 318 records - same on HV2



#perguntar ao professor: nestas variáveis, assumimos que os 0s são missings, incoerências, ou representam casas alugadas.
#decisão final:
#assumption -> there were homes with very low market values, so when the values where divided by 100 to get the value for 
#the column, it was so close to 0 that the int was 0

In [27]:
#HV3 and HV4, related to rents, also have values 0 but that can mean the people are not paying any rent (maybe they bought
#the house already, or maybe they live with family members and are not paying, etc)

In [28]:
#IC1-IC5: values 0 -> don't declare anything to the state?

In [29]:
#data['TPE10'] has values of 0: people that work at home or live very near to the company/office/...

In [30]:
#from RAMNT_3 until RAMNT_24 -> explicado na parte do Tomás

# Data Visualization

In [ ]:
# Define metric and non 
# RETIRAR DATAS

non_metric_vars = data.loc[:,(data.columns=="id") | (np.array(data.dtypes=="object"))]
metric_vars = data.loc[:,(~data.columns.isin(non_metric_vars)) | (np.array(data.dtypes !="datetime"))]

In [ ]:
# Metric Variables Histograms 
sb.set()

# Create individual axes 
fig, axes = plt.subplots(ceil(len(metric_var)/2), 2, figsize=(20, 900))
plt.subplots_adjust(hspace = 0.18)

# Plot data
for ax, feat in zip(axes.flatten(), metric_var):
    ax.hist(data[feat])
    ax.set_title(feat, y = -0.16)
    
plt.show()

In [ ]:
# Non Metric Variables Histograms
sb.set()

# Prepare figure. Create individual axes where each histogram will be placed
fig, axes = plt.subplots(ceil(len(non_metric_var) / 2), 1, figsize=(20, 100))
plt.subplots_adjust(hspace = 0.18)

# Plot data
for ax, feat in zip(axes.flatten(), non_metric_var):
    sb.countplot(x=data[feat],  ax=ax)

plt.show()

In [ ]:
# Metric Variables Boxplots
sb.set(style="whitegrid")

#Prepare figure layout
fig, axes = plt.subplots(len(metric_vars), 1, figsize=(15,8))

# Draw the boxplots
for i in zip(axes, metric_vars):
    sb.boxplot(x="variable", y="value", data=data.loc[data["variable"]==i[1]], ax=i[0])
    i[0].set_xlabel("")
    i[0].set_ylabel("")

# Finalize the plot
plt.suptitle("Metric variables' box plots", fontsize=25)
sb.despine(bottom=True)
plt.show()

In [ ]:
# Metric Variables Correlation Matrix
sb.set(style="white")

# Compute the correlation matrix
corr = metric_vars.corr() #Getting correlation of numerical variables

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool) #Return an array of zeros (Falses) with the same shape and type as a given array
mask[np.triu_indices_from(mask)] = True #The upper-triangle array is now composed by True values

# Set up the matplotlib figure
fig, ax = plt.subplots(figsize=(12, 8))

# Generate a custom diverging colormap
cmap = sb.diverging_palette(220, 10, as_cmap=True) #Make a diverging palette between two HUSL colors. Return a matplotlib colormap object.

# Draw the heatmap with the mask and correct aspect ratio
sb.heatmap(corr, mask=mask, cmap=cmap, center=0, square=True, linewidths=.5, ax=ax)

# Layout
plt.subplots_adjust(top=0.95)
plt.suptitle("Correlation matrix", fontsize=25)
plt.yticks(rotation=0)

# Fixing the bug of partially cut-off bottom and top cells
b, t = plt.ylim() # discover the values for bottom and top
b += 0.5 # Add 0.5 to the bottom
t -= 0.5 # Subtract 0.5 from the top
plt.ylim(b, t) # update the ylim(bottom, top) values
plt.show()

In [ ]:
# Metric Variables Correlation Matrix
def color_red_or_green(val):
    if val < -0.7:
        color = 'background-color: red'
    elif val > 0.7:
        color = 'background-color: green'
    else:
        color = ''
    return color

# Interpreting each Principal Component
correlations = data[metric_var].corr()
correlations.style.applymap(color_red_or_green)

## Creation of new variables (before Feature Engineering)

In [1]:
# we need it for the outliers - Dates in days or years, ...

In [52]:
from datetime import datetime
import math

In [88]:
data['ODATEDW_days'] = (datetime(2020, 1, 1) - data['ODATEDW']).dt.days

#AGE needs to be int after being imputed
data['AGE'] = ((datetime(2020, 1, 1) - data['DOB']).dt.days / 365).map(lambda x: math.floor(x) if math.isnan(x) == False else x)

#from ADATE_2 until ADATE_24 -> these will be droped

data['MAXADATE_days'] = (datetime(2020, 1, 1) - data['MAXADATE']).dt.days

#from RDATE_3 until RDATE_24 -> these will be droped

data['MINRDATE_days'] = (datetime(2020, 1, 1) - data['MINRDATE']).dt.days

data['MAXRDATE_days'] = (datetime(2020, 1, 1) - data['MAXRDATE']).dt.days

data['LASTDATE_days'] = (datetime(2020, 1, 1) - data['LASTDATE']).dt.days

data['FISTDATE_days'] = (datetime(2020, 1, 1) - data['FISTDATE']).dt.days

data['NEXTDATE_days'] = (datetime(2020, 1, 1) - data['NEXTDATE']).dt.days

## Coherence Check

In [41]:
import pandas as pd
import numpy as np
import datetime
import os

In [42]:
#pd.set_option('display.max_columns', None)
path = os.path.join('..', 'data', 'donors.csv')

donors = pd.read_csv(path, sep = ',')

pd.set_option('display.max_columns', None)

D:\Programas\Anaconda\envs\dmi\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (9,412) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [43]:
donors.columns.tolist()

['Unnamed: 0',
 'ODATEDW',
 'OSOURCE',
 'TCODE',
 'STATE',
 'ZIP',
 'MAILCODE',
 'PVASTATE',
 'DOB',
 'NOEXCH',
 'RECINHSE',
 'RECP3',
 'RECPGVG',
 'RECSWEEP',
 'MDMAUD',
 'DOMAIN',
 'HOMEOWNR',
 'CHILD03',
 'CHILD07',
 'CHILD12',
 'CHILD18',
 'NUMCHLD',
 'INCOME',
 'GENDER',
 'WEALTH1',
 'HIT',
 'MBCRAFT',
 'MBGARDEN',
 'MBBOOKS',
 'MBCOLECT',
 'MAGFAML',
 'MAGFEM',
 'MAGMALE',
 'PUBGARDN',
 'PUBCULIN',
 'PUBHLTH',
 'PUBDOITY',
 'PUBNEWFN',
 'PUBPHOTO',
 'PUBOPP',
 'DATASRCE',
 'MALEMILI',
 'MALEVET',
 'VIETVETS',
 'WWIIVETS',
 'LOCALGOV',
 'STATEGOV',
 'FEDGOV',
 'SOLP3',
 'SOLIH',
 'MAJOR',
 'WEALTH2',
 'GEOCODE',
 'COLLECT1',
 'VETERANS',
 'BIBLE',
 'CATLG',
 'HOMEE',
 'PETS',
 'CDPLAY',
 'STEREO',
 'PCOWNERS',
 'PHOTO',
 'CRAFTS',
 'FISHER',
 'GARDENIN',
 'BOATS',
 'WALKER',
 'KIDSTUFF',
 'CARDS',
 'PLATES',
 'LIFESRC',
 'PEPSTRFL',
 'POP901',
 'POP902',
 'POP903',
 'POP90C1',
 'POP90C2',
 'POP90C3',
 'POP90C4',
 'POP90C5',
 'ETH1',
 'ETH2',
 'ETH3',
 'ETH4',
 'ETH5',
 'ETH6',
 'E

# start


In [44]:
#BELO - COHERENCE
#it has more than 50 states bc it includes US territories like the Virgin Islands, Northern Mariana Islands, Guam, etc
donors['STATE'].nunique()

57

In [45]:
#no donors after today's date
donors['ODATEDW'] = pd.to_datetime(donors['ODATEDW'])
(donors['ODATEDW'] > datetime.datetime.now()).sum()

0

In [46]:
donors['NOEXCH'].value_counts()
#drop this! (incoherent, bc the variable is not very relevant and it 
#is not consistent with the metadata's variables explanation -> explicar

#melhor no relatório)

0    61203
0    33882
1      195
1       90
X       35
         7
Name: NOEXCH, dtype: int64

In [47]:
donors['GENDER'].value_counts() 
# A and C are incoherent, we will assume as U: Unknown

F    51277
M    39094
      2957
U     1715
J      365
A        2
C        2
Name: GENDER, dtype: int64

In [48]:
print('< 0: \n', (donors.iloc[:, 41:47] < 0).sum())
print('> 100: \n', (donors.iloc[:, 41:47] > 100).sum())

#no incoherence

< 0: 
 MALEMILI    0
MALEVET     0
VIETVETS    0
WWIIVETS    0
LOCALGOV    0
STATEGOV    0
dtype: int64
> 100: 
 MALEMILI    0
MALEVET     0
VIETVETS    0
WWIIVETS    0
LOCALGOV    0
STATEGOV    0
dtype: int64


In [49]:
#PROBLEM on AGE902: "Median Age of Adults 18 or Older" but it had a value that was 0... actually, after analyzing the 
#subset of donors that had AGE902 equal to 0, we found out that the vast majority of them had also 0 in basically all
#socio-demographic variables. This leads tu belive the missing values on this columns may be represented by a 0 other than
#a Numpy NaN. Anyways, these records have incoherences, and for this reason, they should be removed (799 records!).

#They have donations... but they should be removed anyways.
#dps disto, o MSA, ADI e DMA deixam de ter missings!

#after removing this records other incoherences in other variables were also removed

data = data[data['AGE902'] != 0]

In [50]:
donors['AGE906'].min()

27

In [52]:
(donors.iloc[:, 75:358]).min()
#(donors.iloc[:, 75:358]).max()

## percentagens sem incoerencias (tudo entre 0-10), nao sei se precisamos incluir


#  Minimos:
#HHP1/2 == 0 doenst make sense
#HV1/2 a house with 0 value ??
#4/5 rent == 0 ?? 
#RPH's dont make sense to have avg of 0 rooms per housing unit 
#IC1:5 also doesnt make sense we have avg and medians of value 0
#TPE10/11 MEDIAN == 0 ??
#EC1 lowest values == 89 ?? doesnt make any sense 
#MHUC1/2 also minimum value of 0

#most of median minimum values == 0 doesnt make sense.. DROP?????


# Maximos:
# HHP1/2 == 650/700 how is that possible??
# RPH's value are too high 85/90/61/40
## EC1 -- 170 years of median, impossible
#MHUC1/2 max value 21/5 values are too low 


#I think most or all median values are incorrect. 
#Some are impossible and other are possible but dont make sense I a real world scenario
#for that I think they should be removed and use only the percentages for our analysis

POP903     0.0
POP90C1    0.0
POP90C2    0.0
POP90C3    0.0
POP90C4    0.0
          ... 
HC20       0.0
HC21       0.0
MHUC1      0.0
MHUC2      0.0
AC1        0.0
Length: 283, dtype: float64

In [53]:
#PROBLEM on HV1: "Median Home Value in hundreds" has values of 0, and then, the second lowest, 75 (which means 7500)... 
#the values of 0 are incoherences!!? (Belo) (318 records - same on HV2)


In [54]:
#EC1: "Median Years of School Completed by Adults 25+" has minimum 89 and maximum 170... are the values wrong?

#HC1: "Percent Median Length of Residence" is percent and median at the same time? And has values of 0 length! Is it the 
#median time someone has been living in their current home, in years? Very ambiguous:
#HC2: this one has a less ambiguous description but it still says "Percent Median" so is it a percentage or a median? 
#it is confusing

In [55]:
donors['ODATEDW'].head()

0   2009-01-01
1   2014-01-01
2   2010-01-01
3   2007-01-01
4   2006-01-01
Name: ODATEDW, dtype: datetime64[ns]

In [87]:
#now the dates are datetime
donors['FISTDATE'] = pd.to_datetime(donors['FISTDATE'], errors='ignore')

#            FISTDATE:  Date of first gift 
#                cant be before 
#   ODATEDW: Origin Date. Date of donor's first gift to PVA

#                  RIGHT ?!!!!!!!!!!!!!????????????

(donors['FISTDATE'] < donors['ODATEDW']).sum()

## FISTDATE < ODATEDW substituir por ODATEDW - incoerencia

17445

In [57]:
donors['ODATEDW'] = pd.to_datetime(donors['ODATEDW'])
donors['DOB'] = pd.to_datetime(donors['DOB'], errors='coerce')
donors.iloc[:, 350:373] = donors.iloc[:, 350:373].apply(pd.to_datetime, errors='coerce') #from ADATE_2 until ADATE_24
donors['MAXADATE'] = pd.to_datetime(donors['MAXADATE'])
donors.iloc[:, 378:400] = donors.iloc[:, 378:400].apply(pd.to_datetime, errors='coerce') #from RDATE_3 until RDATE_24
donors['MINRDATE'] = pd.to_datetime(donors['MINRDATE'])
donors['MAXRDATE'] = pd.to_datetime(donors['MAXRDATE'])
donors['LASTDATE'] = pd.to_datetime(donors['LASTDATE'])
donors['FISTDATE'] = pd.to_datetime(donors['FISTDATE'], errors='coerce')
donors['NEXTDATE'] = pd.to_datetime(donors['NEXTDATE'], errors='coerce')

###### DOB incoherence

In [58]:
#DOB incoherence
#Assuming we are in 2020 we have donors with less than 18 years (we dont have nothing saying that they must be 18
#so we will keep the variables). What we are going to check is if they have children if they are less than 16

(donors.loc[:, 'DOB'] > datetime.datetime(2020, 1, 1)).sum()
#No DOB after 2020-1-1


0

In [59]:
donors.DOB.isnull().sum()
#24k MV's

23488

In [60]:
(donors.loc[:, 'DOB'] > datetime.datetime(2004, 12, 31)).sum()
#we have 424 donors with less than 16 years

424

In [68]:
donors.loc[(donors['DOB']> datetime.datetime(2004, 12, 31)) & (donors['NUMCHLD'] > 0), ['DOB', 'NUMCHLD', 'GENDER']]
#At least 24 children under 16 yo have 1 or more child... drop??

,DOB,NUMCHLD,GENDER
6892,2010-09-01,1.0,M
7214,2010-09-01,1.0,F
10687,2007-03-01,3.0,M
11591,2010-02-01,1.0,M
19864,2010-03-01,4.0,F
25211,2010-09-01,1.0,M
25335,2010-08-01,2.0,M
25598,2010-05-01,1.0,M
37897,2010-09-01,1.0,F
40851,2010-06-01,1.0,F


##### NUMCHLD INCOHERENCE

In [62]:
donors['CHILD03'].unique()

array([' ', 'M', 'F', 'B'], dtype=object)

In [63]:
donors['CHILD07'].unique()

array([' ', 'M', 'B', 'F'], dtype=object)

In [64]:
donors['CHILD12'].unique()

array([' ', 'F', 'M', 'B'], dtype=object)

In [65]:
donors['CHILD18'].unique()

array([' ', 'M', 'F', 'B'], dtype=object)

In [66]:
#Assuming that when we have: ' ' the donors has no child

(donors.loc[(donors['CHILD03'] == ' ') & (donors['CHILD07'] == ' ') & (donors['CHILD12'] == ' ') & (donors['CHILD18'] == ' '),
           'NUMCHLD'])

0        NaN
2        NaN
3        NaN
4        1.0
5        NaN
        ... 
95406    NaN
95407    NaN
95409    NaN
95410    NaN
95411    NaN
Name: NUMCHLD, Length: 88742, dtype: float64

In [67]:
donors['NUMCHLD'].isnull().sum()

82283

In [79]:
donors.replace({'OSOURCE' : {' ' : np.nan},
'MAILCODE' : {' ' : '0', 'B' : '1'},
'PVASTATE' : {' ' : 'N'}, #N = state is not served by any organization
'RECINHSE' : {' ' : '0', 'X' : '1'},
'RECP3' : {' ' : '0', 'X' : '1'},
'RECPGVG': {' ' : '0', 'X' : '1'},
'RECSWEEP': {' ' : '0', 'X' : '1'},
'HOMEOWNR' : {' ' : '0', 'H' : '1', 'U' : '0'}, #we assume the blank space on this variable says that it is unknown wether the donor has or not a home (so, we joined it with the 0-Unkwown)
'CHILD03' : {' ' : 'N'}, #the ' ' (blank) will be replaced by 'N' (N = None), as we assume the blank space in this variable means the donor has no
'CHILD07' : {' ' : 'N'}, #children in that specific range of ages
'CHILD12': {' ' : 'N'}, #note: more than 90 000 records have None, on each of these 4 variables
'CHILD18': {' ' : 'N'},
'NUMCHLD': {np.nan : 0}, #assuming NaN is zero, bc there is no zero on this variable!?
'GENDER' : {' ' : 'U', 'A' : 'U', 'C' : 'U'}, #Incoherence (Belo!) -> nem que se fale só no relatório!
#values 'A' (2 of them) & 'C' (2) are not described on the metadata, most probably those are errors
#we assume the blank spaces, As and Cs can be assumed to be Unkwown
'SOLP3' : {' ' : 'Default'},
'SOLIH' : {' ' : 'Default'}, #feature engineering: create a binary with -> 0 (can't be mailed), 1 (can be mailed)
'MAJOR' : {' ' : '0', 'X' : '1'},
'GEOCODE2' : {' ' : np.nan}}, inplace = True)

In [70]:
donors['NUMCHLD'].unique() #assuming nan is zero childs

array([nan,  1.,  2.,  3.,  4.,  5.,  6.,  7.])

In [82]:
(donors.loc[(donors['CHILD03'] != 'N') | (donors['CHILD07'] != 'N') | (donors['CHILD12'] != 'N') | (donors['CHILD18'] != 'N')
            | (donors['NUMCHLD'] != 0), 'NUMCHLD'])

1        1.0
4        1.0
6        1.0
10       0.0
12       2.0
        ... 
95373    0.0
95375    1.0
95377    2.0
95404    1.0
95408    1.0
Name: NUMCHLD, Length: 13869, dtype: float64

In [ ]:
#New column CHILD: 1 has children(filhos) 0 cc

